In [51]:
from pathlib import Path
import os
import pandas as pd
from glob import glob

In [2]:
# Check Your Working Directory
print('Current working directory:', os.getcwd())

Current working directory: f:\datascience\datascience_project\project\new2025\github\Deploy_chest_X-ray_pneumonia\notebooks


In [4]:
%cd ..

f:\datascience\datascience_project\project\new2025\github\Deploy_chest_X-ray_pneumonia


In [14]:
data_map=pd.read_csv('data/data_map.csv')
data_map.head()







,img-path,class-name,split-name,resized_img
0,data\chest_xray\chest_xray\test\NORMAL\IM-0001...,NORMAL,test,data/resize_image/test_NORMAL_IM-0001-0001.jpeg
1,data\chest_xray\chest_xray\test\NORMAL\IM-0003...,NORMAL,test,data/resize_image/test_NORMAL_IM-0003-0001.jpeg
2,data\chest_xray\chest_xray\test\NORMAL\IM-0005...,NORMAL,test,data/resize_image/test_NORMAL_IM-0005-0001.jpeg
3,data\chest_xray\chest_xray\test\NORMAL\IM-0006...,NORMAL,test,data/resize_image/test_NORMAL_IM-0006-0001.jpeg
4,data\chest_xray\chest_xray\test\NORMAL\IM-0007...,NORMAL,test,data/resize_image/test_NORMAL_IM-0007-0001.jpeg


In [52]:
from sklearn.preprocessing import LabelEncoder


In [9]:
# Convert text labels (NORMAL, PNEUMONIA) to numbers (0, 1) for machine learning
data_map['class-encoded']= LabelEncoder().fit_transform(data_map['class-name'])
data_map.head()

,img-path,class-name,split-name,resized_img,class-encoded
0,data\chest_xray\chest_xray\test\NORMAL\IM-0001...,NORMAL,test,data/resize_image/test_NORMAL_IM-0001-0001.jpeg,0
1,data\chest_xray\chest_xray\test\NORMAL\IM-0003...,NORMAL,test,data/resize_image/test_NORMAL_IM-0003-0001.jpeg,0
2,data\chest_xray\chest_xray\test\NORMAL\IM-0005...,NORMAL,test,data/resize_image/test_NORMAL_IM-0005-0001.jpeg,0
3,data\chest_xray\chest_xray\test\NORMAL\IM-0006...,NORMAL,test,data/resize_image/test_NORMAL_IM-0006-0001.jpeg,0
4,data\chest_xray\chest_xray\test\NORMAL\IM-0007...,NORMAL,test,data/resize_image/test_NORMAL_IM-0007-0001.jpeg,0


In [104]:
import  torch 
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms ,models
from PIL import Image




In [105]:
class pneumonia_dataset(Dataset):
    """
        class is defined to handle loading individual images and
          their corresponding labels based on the data map
        
        Args:
            df_split (pd.DataFrame): DataFrame containing labels
            transform (callable, optional): Optional transform to be applied to images
        """
        
    def __init__(self, df_split,transform):

     self.df_split= df_split
     self.transform = transform
      

    def __len__(self):
        """return the number of images in dataset for data loader

       """
        return len (self.df_split)
    
    def __getitem__(self,index) :
        """
        Load an image and its corresponding label at a specific index
        return a tensor of the image and the label 
        
   """     
        # Get the image path from the dataframe at current index
        img_path = self.df_split.iloc[index]['resized_img']
        # Get the label (0 or 1) from the dataframe at current index
        label = self.df_split.iloc[index]['class-name']
        # Convert label to numeric using LabelEncoder
        label_encoder = LabelEncoder()
        label = label_encoder.fit_transform([label])[0]
        # Open and load the image file and convert to grayscale
        image= Image.open(img_path).convert('L')
        # Apply the transforms to convert image to tensor and normalize it 
        image_tensor= self.transform(image)
        # Return both the transformed image and its label
        return  image_tensor,label
        
        


In [69]:
# Create a training dataset by:
# 1. Filtering data_map to get only training data (where split-name is 'train')
# 2. Converting images to tensors using transforms.ToTensor()
# 3. Creating a pneumonia_dataset object that will load images one by one
train_dataset= pneumonia_dataset(data_map[data_map['split-name']=='train'],transform= transforms.ToTensor())


In [106]:
# Create a validation dataset by:
# 1. Filtering data_map to get only validation data (where split-name is 'val')
# 2. Converting images to tensors using transforms.ToTensor()
# 3. Creating a pneumonia_dataset object that will load images one by one
val_dataset=pneumonia_dataset(data_map[data_map['split-name']== 'val'],transform= transforms.ToTensor())

# Get the first image and label from validation dataset to verify loading works
val_dataset[0]


(tensor([[[0.1451, 0.1137, 0.1020,  ..., 0.3529, 0.3020, 0.2549],
          [0.0980, 0.0784, 0.0824,  ..., 0.3412, 0.2902, 0.2431],
          [0.0353, 0.0353, 0.0588,  ..., 0.3294, 0.2745, 0.2275],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]]),
 np.int64(0))

In [ ]:
train_dataset[0]

(tensor([[[0.0863, 0.0824, 0.0784,  ..., 0.3765, 0.3373, 0.3725],
          [0.0863, 0.0784, 0.0667,  ..., 0.3647, 0.3569, 0.3608],
          [0.0863, 0.0745, 0.0510,  ..., 0.3490, 0.3765, 0.3451],
          ...,
          [0.1020, 0.1059, 0.1059,  ..., 0.1176, 0.1216, 0.1255],
          [0.1176, 0.1176, 0.1176,  ..., 0.2314, 0.2353, 0.2392],
          [0.1569, 0.1569, 0.1569,  ..., 0.3765, 0.3804, 0.3843]]]),
 np.int64(0))

In [109]:
# Create data loaders for training and validation datasets
# Using batch size of 4  to avoid crashing and enabling shuffling for both
train_loader = DataLoader(train_dataset,batch_size=4,shuffle= True)
val_dataset = DataLoader(val_dataset, batch_size=4,shuffle=True)

### Using ResNet18 pre-trained model because:
 1. It's a proven architecture that performs well on image classification tasks
 2. Pre-trained on ImageNet means it already learned useful image features
 3. Relatively lightweight compared to deeper ResNets (18 layers vs 50+ in other variants)
 4. Transfer learning from ImageNet features helps with medical image analysis
 5. Small enough to run on limited GPU resources while training

 Download pre-trained ResNet18 model from torchvision  
 We'll use it as our starting point and fine-tune it for chest X-ray classification
 weights=models.ResNet18_Weights.IMAGENET1K_V1 specifies that we want to use the pre-trained weights  
 from the ResNet18 model that was trained on ImageNet dataset version 1   
 This allows us to leverage transfer learning by starting with a model that has already
 learned useful image features from the massive ImageNet dataset

In [72]:
model = models.resnet18(weights= models.ResNet18_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Electronica Care/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:24<00:00, 1.93MB/s]


ResNet18 is designed for 3-channel color images and typically outputs probabilities for 1000 classes30.   
Since this project uses grayscale (1 channel) images and has only two output classes, the first layer (model.conv1) is modified to accept 1 input channel instead of 3, and the final fully connected layer (model.fc) is modified to output 2 values instead of 100030.  
 If the first layer is changed, it's generally recommended to retrain the entire model31.
  2D convolution expecting 4 dimensions for batch, channels, height, width

In [81]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [83]:
model.conv1 = nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3,bias=False)
model.fc= nn.Linear(in_features=512,out_features=2,bias=True)


GPU Training (CUDA): To speed up training, the model and the data batches are moved to a GPU device using model.to(device) and tensor.to(device)  
The device is determined based on whether CUDA (NVIDIA GPU support) is available

In [76]:
#the standard PyTorch practice to write portable code that works on both GPU and CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Optimizer and Loss: An Optimizer (like Adam) is defined to update the model's weights during training32.    
 A Loss Function (like CrossEntropyLoss for classification) is defined to measure the error between the model's predictions and the true labels

In [77]:
# Create an optimizer using Adam algorithm with learning rate 0.001
# This helps the model learn by adjusting its parameters
optimizer= torch.optim.Adam(model.parameters(),lr=0.001)

# This measures how wrong the model's predictions are compared to correct answers
loss_fn = nn.CrossEntropyLoss()

In [78]:
from tqdm import tqdm

Training Loop: The core training happens in a loop over a specified number of epochs (passes through the full dataset)   
 Inside the epoch loop, it iterates through batches of data from the DataLoader  
  For each batch: gradients are zeroed (optimizer.zero_grad()), the forward pass is computed (getting model outputs),  
   the loss is calculated, the backward pass (loss.backward()) computes gradients, and the optimizer updates weights (optimizer.step())   
 Running loss is tracked

In [112]:
for epoch in range(1):
    # Put model in training mode
    model.train()
    # Keep track of total loss for this epoch
    running_loss=0.0
    
    # Loop through batches of images and their labels, with a progress bar
    for imgs,labels in tqdm(train_loader,total=len(train_loader)):
        # Move images and labels to GPU/CPU device
        imgs= imgs.to(device)
        labels= labels.to(device)
        
        # Clear gradients from previous batch
        optimizer.zero_grad()
        
        # Get model's predictions for this batch
        preds =model(imgs)
        
        # Calculate how wrong the predictions were
        loss_val=loss_fn(preds,labels)
        
        # Calculate gradients for model parameters
        loss_val.backward()
        
        # Update model parameters using gradients
        optimizer.step()
        
        # Add this batch's loss to running total
        running_loss += loss_val.item()
        
        # Print progress update showing average loss so far
        print(f"Epoch{epoch+1} of 1, loss:{running_loss/len(train_loader)}")
        


  0%|          | 1/1304 [00:02<1:01:41,  2.84s/it]

Epoch1 of 1, loss:1.2813469113287818e-07


  0%|          | 2/1304 [00:03<31:08,  1.43s/it]  

Epoch1 of 1, loss:2.555610253861277e-07


  0%|          | 3/1304 [00:03<21:55,  1.01s/it]

...

Epoch1 of 1, loss:0.00012131114387551738


100%|█████████▉| 1303/1304 [12:03<00:00,  2.01it/s]

Epoch1 of 1, loss:0.00012137744203303616


100%|██████████| 1304/1304 [12:04<00:00,  1.80it/s]

Epoch1 of 1, loss:0.00012144248329677571


In [113]:
# Save the trained model weights to a file
torch.save(model.state_dict(),"models/model.pth")